In [2]:
import numpy as np 
import pandas as pd 
import sqlite3 as sql 
import json
from tqdm import tqdm 
import datetime as dt


fname_daily = '../../../data/prices/stocks.db'
fname_intra = '../../../data/prices/stocks_intraday.db'
fname_prices = '../../../data/prices/stocks.db'
stocks_json = '../../../data/stocks/tickers.json'
 
stocks = json.load(open(stocks_json, 'r'))['all_stocks']
daily_conn = sql.connect(fname_daily)
intra_conn = sql.connect(fname_intra)
price_conn = sql.connect(fname_prices)

In [4]:
import sys 
sys.path.append('../')
from indicators import Indicators


def time_splits(symbol, column = ['Close'], db_path = 'data/prices/stocks.db'):
    ''' Returns a dataframe of indicators for each time split. 
            If Daily time frame: Split into 3d, 1w, 1m, and 3m.
            If Intraday time frame: Split into 5m, 15m, 30m, and 1h.
    '''
    df = pd.read_sql(f'select * from {symbol}', sql.connect(db_path), parse_dates = ['Date'], index_col='Date')
    
    if df.index[1] - df.index[0] < pd.Timedelta('1 day'):
        time_splits = ['5min', '15min', '30min', '1h']
        lodf = [df.resample(t).last().dropna() for t in time_splits]
    else:
        time_splits = ['3d','1w', '1m', '3m']
        lodf = [df.resample(t).last().dropna() for t in time_splits]
    return dict(zip(time_splits, lodf))

# def time_splits(syms,  column = ['Close'], db_path = 'data/prices/stocks.db'):
#     ''' Loops over each symbol and returns the time splits. '''
#     return {s: indicator_time_splits(s, db_path = db_path) for s in syms}


syms = ['SPY', 'QQQ']
print('\n\nTime Splits\n\n')


lodf = time_splits(syms[0], db_path = fname_intra)
df = lodf['5min']
df.columns = [c.lower() for c in df.columns]



Time Splits




In [5]:
df

,adj close,close,high,low,open,volume
Date,,,,,,
2023-08-10 09:30:00,449.329987,449.329987,449.369995,448.779999,448.799988,222954.0
2023-08-10 09:35:00,449.700012,449.700012,449.989990,449.429993,449.875000,593492.0
2023-08-10 09:40:00,449.600006,449.600006,449.630005,449.385010,449.410004,175954.0
2023-08-10 09:45:00,450.410004,450.410004,450.559998,450.339996,450.545013,492409.0
2023-08-10 09:50:00,450.665009,450.665009,450.670013,450.420013,450.665009,140464.0
...,...,...,...,...,...,...
2024-05-24 12:20:00,529.830017,529.830017,529.830017,529.770020,529.780518,14858.0
2024-05-24 12:25:00,529.789978,529.789978,529.789978,529.710022,529.760010,39339.0
2024-05-24 12:30:00,529.679993,529.679993,529.713928,529.620117,529.680115,25727.0


In [6]:
from ta_indicators import Indicators

ind = Indicators(df)
indicator_df = ind.get_indicators()
indicator_df.tail()


,adj close,close,high,low,open,volume,stoch_rsi,rsi,kama,sma_slow,...,bb_percent,kc_percent,atr,macd,adx,recent_high,recent_low,all_time_high,all_time_low,cum_returns
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-24 12:20:00,529.830017,529.830017,529.830017,529.770020,529.780518,14858.0,0.353156,61.453072,529.356030,528.849151,...,0.657813,1.569215,0.299696,0.455380,23.196585,530.210022,528.470520,531.859985,409.350006,0.169831
2024-05-24 12:25:00,529.789978,529.789978,529.789978,529.710022,529.760010,39339.0,0.324466,60.680363,529.403830,528.902150,...,0.617431,1.242925,0.286860,0.437024,22.774264,530.210022,528.768677,531.859985,409.350006,0.169756
2024-05-24 12:30:00,529.679993,529.679993,529.713928,529.620117,529.680115,25727.0,0.195449,58.504185,529.410088,528.960721,...,0.508690,0.600059,0.278503,0.408888,22.121270,530.210022,529.030029,531.859985,409.350006,0.169548
2024-05-24 12:35:00,529.429871,529.429871,529.580017,529.390015,529.539978,32710.0,0.000000,53.780578,529.410571,529.003917,...,0.250812,-0.504690,0.279323,0.362232,20.873409,530.210022,529.151001,531.859985,409.350006,0.169076
2024-05-24 12:40:00,529.530090,529.530090,529.530090,529.530090,529.530090,0.0,0.173476,55.336630,529.411610,529.063634,...,0.357187,-0.129680,0.266530,0.329545,19.714681,530.210022,529.151001,531.859985,409.350006,0.169265
